# DeFi LP Landscape Update

## 1. Setup

### Install and import all relevant libraries

In [47]:
%%capture
!pip install gspread
!pip install pandas
!pip install gspread-dataframe
!pip install oauth2client
!pip install PyDrive

In [48]:
# Import all relevant libraries
import pandas as pd
import json
import time
from datetime import datetime as dt
import csv
import requests
import gspread
import gspread_dataframe as gd
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from collections import defaultdict

### Connect to Google Sheets API and setup Defillama API

In [49]:
# Set up Google Sheets API credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('lp-landscape-analysis-dd6d6479b244.json', scope)
gc = gspread.authorize(credentials)

In [50]:
# API TVL base URL
tvl_base_url = 'https://api.llama.fi'

# API Yields base URL
yields_base_url = 'https://yields.llama.fi'

## 2. Update Data in 'DeFi Landscape LP Opportunities'

### Helper functions

In [51]:
# HELPER FUNCTIONS

# Writes a list of values to a column of the input sheet starting at starting_cell
def write_to_column(sheet, starting_cell, values):
  sheet.update(starting_cell, [[i] for i in values], value_input_option="USER_ENTERED")
  print("Values written to sheet successfully")

# Converts a dataframe column from str to date format in-place
def df_str_to_date(df, column_name, date_format):
  df[column_name] = pd.to_datetime(df[column_name], format=date_format, errors='raise')
  df[column_name] = df[column_name].dt.date

# Write list to CSV
def write_list_to_csv(filename, list):
   # Open the file in write mode
  with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Writing each item in the list to the file
    for element in list:
        writer.writerow([element])


### Open spreadsheet and load relevant tabs

In [52]:
# Open the Google Sheet we'll be reading and writing to
lp_landscape = gc.open('DeFi Landscape LP Opportunities_v2')

In [53]:
# Select the 'Project Ratings' tab and read the data into a dataframe
project_ratings_sheet = lp_landscape.worksheet("Project Ratings")
lp_project_ratings = project_ratings_sheet.get_all_records()
lp_project_ratings_df = pd.DataFrame(lp_project_ratings)

# Select the 'Stables' tab and read the data into a dataframe
stables_sheet = lp_landscape.worksheet("Stables")
lp_stables = stables_sheet.get_all_records()
lp_stables_df = pd.DataFrame(lp_stables)

# Select the 'Volatile' tab and read the data into a dataframe
volatile_sheet = lp_landscape.worksheet("Volatile")
lp_volatile = volatile_sheet.get_all_records()
lp_volatile_df = pd.DataFrame(lp_volatile)

# Select 'Pool Yields' tab
pool_yields_sheet = lp_landscape.worksheet("Pool Yields")

# Select 'Protocol Historicals' tab
protocol_historicals_sheet = lp_landscape.worksheet("Protocol Historicals")

# Select 'Pool Historicals' tab
pool_historicals_sheet = lp_landscape.worksheet("Pool Historicals")

# Select 'Protocol Info' tab
protocol_info_sheet = lp_landscape.worksheet("Protocol Info")

# Select 'Historical Chain TVL' tab
historical_chain_tvl_sheet = lp_landscape.worksheet("Historical Chain TVL")

### 2.1. Update data in 'Project Ratings' tab

In [54]:
# Extract protocol names from the worksheet as a list in the format used by Defillama API (slug)
protocol_slugs = lp_project_ratings_df["API Protocol Name"].tolist()
protocol_slugs = [protocol for protocol in protocol_slugs if len(protocol) > 1]

In [55]:
# Write list to csv file
write_list_to_csv("protocol_slugs.csv", protocol_slugs)

#### 2.1.1. Update current protocol TVLs

In [56]:
# Get current TVLs for the list of protocols and return them as a list
protocol_tvls = [requests.get(tvl_base_url + '/tvl/' + protocol).json() for protocol in protocol_slugs]

# Format numbers as millions before writing to spreadsheet
protocol_tvls_m = [int(tvl) / 1000000 for tvl in protocol_tvls]

In [57]:
# Write results stored in protocol_tvls to 'Current TVL ($m)' in 'Project Ratings' tab to update current protocol TVLs
write_to_column(project_ratings_sheet, "C2", protocol_tvls_m)

/var/folders/vf/7r30jcsd0190g5bqdf7ryl800000gn/T/ipykernel_82881/640623663.py:5: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  sheet.update(starting_cell, [[i] for i in values], value_input_option="USER_ENTERED")


Values written to sheet successfully


#### 2.1.2. Update 1-year average protocol TVLs

In [59]:
# Pull historical TVL data for each protocol in the list broken down by token and chain
protocol_historicals = [requests.get(tvl_base_url + '/protocol/' + protocol).json() for protocol in protocol_slugs]

# List of keys we want to keep
fields = ['name','currentChainTvls']

##### Sanity check to compare aggregated TVL to actual current TVL shown by Defillama

In [60]:
# Create a list of dictionaries with protocol name as the key as current TVLs
# by chain as the value for every protocol in the list

# List to store broken down results
protocol_tvls_by_chain = []

# List to store aggreagated TVL results
aggregated_protocol_tvls = []

# Iterate protocol list
for protocol in protocol_historicals:
  # keep only relevant keys
  p = dict((k, protocol[k]) for k in fields if k in protocol)

  # Create key,value pair from the values associated with the two keys left
  p = {protocol['name']: protocol['currentChainTvls']}

  # Iterate nested dict and remove unwanted chains to prevent double counting TVL
  for k,v in p.items():
    for x in list(v.keys()):
      if "borrowed" in x or "staking" in x or 'pool2' in x:
        del v[x]
    aggregated = (k, sum(v.values()))

  # Add protocol to result list
  protocol_tvls_by_chain.append(p)

  # Add protocol overall current TVL to aggregated list
  aggregated_protocol_tvls.append(aggregated)

In [61]:
# COMPARE AGGREGATED VS. ACTUAL TVLs

current_aggregated_protocol_tvls = [x[1] for x in aggregated_protocol_tvls]
protocol_names = [x[0] for x in aggregated_protocol_tvls]

# Create tuples for each protocol comparing aggregated TVL to TVL shown by DL
protocol_tvl_deltas = list(zip(protocol_names, zip(current_aggregated_protocol_tvls, protocol_tvls)))

In [62]:
protocol_tvl_deltas

[('AAVE', (8399486847.121011, 8653136261.547512)),
 ('Curve Finance', (2333223915.0743003, 2384855534.388074)),
 ('Compound V2', (1034400391.4292, 1040299204.7071826)),
 ('Compound V3', (1421582035.67243, 1431532987.834271)),
 ('Convex Finance', (1743387706.93097, 1736743399.9388666)),
 ('Reserve', (34083870.45423, 34130717.33080939)),
 ('MakerDAO', (6396112284.38318, 6392081481.197919)),
 ('Spark', (3159513792.12362, 3197883161.030174)),
 ('Yearn Finance', (341423602.21011, 343185426.4828875)),
 ('Frax Finance', (1201588509.03096, 1249201605.864286)),
 ('Goldfinch', (4123269.41291, 4138497.7044126242)),
 ('Balancer', (1105351481.14655, 1106200278.458088)),
 ('Flux Finance', (20895651.1826, 20901091.24959967)),
 ('Summer.fi', (3196150654.50773, 3196150654.5077276)),
 ('Stargate', (303236803.87854004, 304258600.30701023)),
 ('Aura', (576380631.27292, 580698087.7371308)),
 ('Abracadabra', (143203155.63134998, 144516042.36201513)),
 ('Velodrome', (124683270.46726, 129771333.46004833)),
 (

##### Aggregate historical TVL broken down by token and chain to get overall historical protocol TVLs

In [63]:
# Create a list of dictionaries with protocol name as the key and historical TVL
# by token and chain as the value for every protocol in the list

# List to store broken down results
clean_protocol_historicals = []

# Iterate protocol list
for protocol in protocol_historicals:
  # Keep only relevant keys
  p = dict((k, protocol[k]) for k in fields if k in protocol)

  # Create key,value pair from the values associated with the two keys left
  p = {protocol['name']: protocol['chainTvls']}

  # Iterate nested dict and remove unwanted chains to prevent double counting TVL
  for k,v in p.items():
    for x in list(v.keys()):
      if "borrowed" in x or "staking" in x or 'pool2' in x:
        del v[x]

  # Add protocol to result list
  clean_protocol_historicals.append(p)

In [64]:
# List to store the final result
historical_protocol_tvls = []

# Aggregate TVL data per date for each chain
for protocol in clean_protocol_historicals:

  # Dictionary to store result
  aggregated_historicals = defaultdict(float)

  for p, d1 in protocol.items():
    for chain, chain_data in d1.items():
        for entry in chain_data["tvl"]:
            date = entry["date"]
            totalLiquidityUSD = float(entry["totalLiquidityUSD"])
            aggregated_historicals[date] += totalLiquidityUSD

  # Convert the aggregated data to a sorted list of tuples
  sorted_aggregated_historicals = sorted(aggregated_historicals.items(), key=lambda x: x[0])

  # Convert the aggregated data to a dataframe
  aggregated_historicals_df = pd.DataFrame(sorted_aggregated_historicals, columns=['date', 'totalLiquidityUSD'])

  # Convert the 'Date' column from UNIX timestamp to datetime format
  aggregated_historicals_df['date'] = pd.to_datetime(aggregated_historicals_df['date'], unit='s')
  aggregated_historicals_df['date'] = aggregated_historicals_df['date'].dt.date

  # Add dataframe to result list
  historical_protocol_tvls.append(aggregated_historicals_df)

In [65]:
# Order by date in reverse chronological order
historical_protocol_tvls = [df.sort_values(by='date', ascending=False) for df in historical_protocol_tvls]

# Create a common index for all dataframes to conserve the order when concatenating
for df in historical_protocol_tvls:
    df.index = range(len(df))

# Concatenate every dataframe in the list into one dataframe
protocol_historicals_df = pd.concat(historical_protocol_tvls, axis = 1, sort=False)

In [66]:
# Insert 1 blank cell between each protocol slug in the list for spreadsheet formatting
formatted_protocol_slugs = []

for slug in protocol_slugs:
  formatted_protocol_slugs.append(slug)
  formatted_protocol_slugs.append("")
formatted_protocol_slugs.pop()

# Clear every cell in the sheet
protocol_historicals_sheet.clear()

# Write formatted list as first row of the 'Protocol Historicals' sheet for indexing
protocol_historicals_sheet.update("A1", [formatted_protocol_slugs])

# Update data in 'Protocol Historicals' sheet under first row
gd.set_with_dataframe(protocol_historicals_sheet, protocol_historicals_df, row=2,
                      include_index=False, include_column_header=True, resize=True)

/var/folders/vf/7r30jcsd0190g5bqdf7ryl800000gn/T/ipykernel_82881/2339619863.py:13: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  protocol_historicals_sheet.update("A1", [formatted_protocol_slugs])


### 2.2 Update 'Pool Yields' raw data tab

In [67]:
# Get all pool yields
yields = requests.get(yields_base_url + '/pools')

# Convert Yields response to data frame
pool_yields_df = pd.DataFrame(yields.json()['data'])

In [68]:
# Clear every cell in the sheet
pool_yields_sheet.clear()

# Update data in 'Pool Yields' sheet
gd.set_with_dataframe(pool_yields_sheet, pool_yields_df, include_index=False,
                      include_column_header=True, resize=True)

### 2.3 Update 'Pool Historicals' raw data tab

In [69]:
# Extract pool ID's from stables worksheet as a list
stable_pool_ids = lp_stables_df["API pool id"].tolist()
stable_pool_ids = [id for id in stable_pool_ids if id]

# Extract pool ID's from volatile worksheet as a list
volatile_pool_ids = lp_volatile_df["API pool id"].tolist()
volatile_pool_ids = [id for id in volatile_pool_ids if id]

# Join both lists
pool_ids = stable_pool_ids + volatile_pool_ids

In [70]:
# Write lists to csv files
write_list_to_csv("stable_pool_ids.csv", stable_pool_ids)
write_list_to_csv("volatile_pool_ids.csv", volatile_pool_ids)
write_list_to_csv("pool_ids.csv", pool_ids)

In [71]:
# Get historical TVL and APY data for each pool and return it as a list of data frames

pool_historicals = []
counter = 0

for id in pool_ids:
    # Keep a counter to prevent API rate limit
    counter += 1

    if counter > 30:
        counter = 0
        # ~1 minute delay to avoid API rate limit issues
        time.sleep(70)

    try:  
        curr = requests.get(yields_base_url + '/chart/' + id).json()['data']
        pool_historicals.append(curr)
    except: 
        print("Error parsing " + str(id))

In [72]:
pool_historicals_dfs = [pd.DataFrame(i).drop(['il7d', 'apyBase7d','apyBase','apyReward'], axis=1) for i in pool_historicals]

# Convert timestamp format from str to date in every data frame
date_format = '%Y-%m-%dT%H:%M:%S.%fZ'

for df in pool_historicals_dfs:
  df_str_to_date(df, 'timestamp', date_format)

# Order by date in reverse chronological order
pool_historicals_dfs = [df.sort_values(by='timestamp', ascending=False) for df in pool_historicals_dfs]

In [73]:
# Create a common index for all dataframes to conserve the order when concatenating
for df in pool_historicals_dfs:
    df.index = range(len(df))

# Concatenate every data frame in the list into one data frame
pool_historicals_df = pd.concat(pool_historicals_dfs, axis = 1, sort=False)

# Insert 4 blank elements between each pool id in the list for spreadsheet formatting
formatted_pool_ids = []

for id in pool_ids:
  formatted_pool_ids.append(id)
  for i in range(2):
    formatted_pool_ids.append("")
formatted_pool_ids.pop()

''

In [74]:
# Clear every cell in the sheet
pool_historicals_sheet.clear()

# Write formatted list as first row of the 'Pool Historicals' sheet for indexing
pool_historicals_sheet.update("A1", [formatted_pool_ids])

# Update data in 'Pool Historicals' sheet under first row
gd.set_with_dataframe(pool_historicals_sheet, pool_historicals_df, row=2,
                      include_index=False, include_column_header=True, resize=True)

/var/folders/vf/7r30jcsd0190g5bqdf7ryl800000gn/T/ipykernel_82881/4049637518.py:5: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  pool_historicals_sheet.update("A1", [formatted_pool_ids])


### 2.4. Update 'Historical Chain TVL' raw data tab

In [75]:
# Get historical TVL data for Ethereum
eth_tvl = requests.get(tvl_base_url + '/v2/historicalChainTvl/Ethereum')

# Convert response to data frame
eth_tvl_df = pd.DataFrame(eth_tvl.json())

# Convert the 'Date' column from UNIX timestamp to datetime format
eth_tvl_df['date'] = pd.to_datetime(eth_tvl_df['date'], unit='s')
eth_tvl_df['date'] = eth_tvl_df['date'].dt.date

# Order by date in reverse chronological order
eth_tvl_df = eth_tvl_df.sort_values(by='date', ascending=False)

In [76]:
# Clear every cell in the sheet
historical_chain_tvl_sheet.clear()

# Update data in 'Historical Chain TVL' sheet
gd.set_with_dataframe(historical_chain_tvl_sheet, eth_tvl_df, include_index=False,
                      include_column_header=True, resize=True)

### 2.5. Update 'Protocol Info' raw data tab

In [77]:
# Get all protocols
protocols = requests.get(tvl_base_url + '/protocols')

# Convert Protocols response to data frame
protocols_df = pd.DataFrame(protocols.json())

In [78]:
# Clear every cell in the sheet
protocol_info_sheet.clear()

# Update data in 'Pool Yields' sheet
gd.set_with_dataframe(protocol_info_sheet, protocols_df, include_index=False,
                      include_column_header=True, resize=True)

### 2.6. Update data in 'LP Update Historicals' tab for LP Landscape Update distribution charts

In [79]:
# HELPER FUNCTIONS

# Aggregate historicals for peer pools, store mean values, and return as dataframe
def aggregate_historicals(all_pools_dict, pool_id_list, exclude_list):
  # Keep only pools not contained in exclude list
  pool_id_list = [id for id in pool_id_list if id not in exclude_list]

  # Keep only data for pools contained in pool_id_list
  peer_pools = {k: all_pools_dict[k] for k in pool_id_list}

  # Separate values (dataframes) from keys (pool id's) before concatenating
  peer_pools_dfs = list(peer_pools.values())

  # Concatenate all dataframes
  peer_pools_df = pd.concat(peer_pools_dfs)

  # Group by timestamp and compute the mean for each group
  aggregated_df = peer_pools_df.groupby('timestamp').mean().reset_index()

  # Order by date in reverse chronological order
  aggregated_df = aggregated_df.sort_values(by='timestamp', ascending=False)

  return aggregated_df

In [80]:
# Create dictionary to associate all pool ids to their historical data
all_pools_dict = dict(zip(pool_ids, pool_historicals_dfs))

In [81]:
# Select the 'Stable Comps' tab and read the data into a dataframe
stable_comps_sheet = lp_landscape.worksheet("Stables Comps")
stable_comps = stable_comps_sheet.get_all_records()
stable_comps_df = pd.DataFrame(stable_comps)

# Select the 'Volatile Comps' tab and read the data into a dataframe
volatile_comps_sheet = lp_landscape.worksheet("Volatile Comps")
volatile_comps = volatile_comps_sheet.get_all_records()
volatile_comps_df = pd.DataFrame(volatile_comps)

In [82]:
# Extract pool names from the worksheet as a list for each index group

base_stable_comps_df = stable_comps_df[stable_comps_df["Chain"] == "Base"]

eth_stable_comps_df = stable_comps_df[stable_comps_df["Chain"] == "Ethereum"]
eth_curve_stable_comps_df = eth_stable_comps_df[eth_stable_comps_df["API project name"] == "curve-dex"]
eth_convex_stable_comps_df = eth_stable_comps_df[eth_stable_comps_df["API project name"] == "convex-finance"]

eth_curve_comps_df = volatile_comps_df[volatile_comps_df["API project name"] == "curve-dex"]
eth_convex_comps_df = volatile_comps_df[volatile_comps_df["API project name"] == "convex-finance"]



base_stable_comps_pool_ids = base_stable_comps_df["API pool id"].tolist()
base_stable_comps_pool_ids = [id for id in base_stable_comps_pool_ids if id]

eth_stable_comps_pool_ids = eth_stable_comps_df["API pool id"].tolist()
eth_stable_comps_pool_ids = [id for id in eth_stable_comps_pool_ids if id]

eth_comps_pool_ids = volatile_comps_df["API pool id"].tolist()
eth_comps_pool_ids = [id for id in eth_comps_pool_ids if id]

In [83]:
# Write lists to csv files
write_list_to_csv("eth_stable_comps_pool_ids.csv", eth_stable_comps_pool_ids)
write_list_to_csv("base_stable_comps_pool_ids.csv", base_stable_comps_pool_ids)
write_list_to_csv("eth_comps_pool_ids.csv", eth_comps_pool_ids)

In [84]:
eth_curve_stable_comps_pool_ids = eth_curve_stable_comps_df["API pool id"].tolist()
eth_curve_stable_comps_pool_ids = [id for id in eth_curve_stable_comps_pool_ids if id]
eth_convex_stable_comps_pool_ids = eth_convex_stable_comps_df["API pool id"].tolist()
eth_convex_stable_comps_pool_ids = [id for id in eth_convex_stable_comps_pool_ids if id]

eth_curve_comps_pool_ids = eth_curve_comps_df["API pool id"].tolist()
eth_curve_comps_pool_ids = [id for id in eth_curve_comps_pool_ids if id]
eth_convex_comps_pool_ids = eth_convex_comps_df["API pool id"].tolist()
eth_convex_comps_pool_ids = [id for id in eth_convex_comps_pool_ids if id]

In [85]:
# Pools to exclude
# HYUSD-EUSD 2faacc5b-7e32-46f3-84e2-061aed8f7f21; c8815168-ba35-4e7c-b7b1-a0b33b6c73bc
# EUSD-FRAXBP c04005c9-7e34-41a6-91c4-295834ed8ac0; 817329d2-07cb-4cbd-82ac-eb9bc0add450
# EUSD-USDC 212375c8-694c-4fa5-8b36-05f50c8e61b2
# ETH+ 4e6cd326-72d5-4680-8d2f-3481d50e8bb1 Include naked LSTs in Comps? 
# ETH+-ETH 5a046093-29fc-4ecb-b90e-daccda151b5b; 74346f6f-c7ee-4506-a204-baf48e13decb
# ETH+/pxETH when gauge goes into effect

In [86]:
# List of pool_ids to exclude from indices (RToken pools)
exclude_pool_ids = ["2faacc5b-7e32-46f3-84e2-061aed8f7f21", "c8815168-ba35-4e7c-b7b1-a0b33b6c73bc",
                    "c04005c9-7e34-41a6-91c4-295834ed8ac0", "817329d2-07cb-4cbd-82ac-eb9bc0add450",
                    "212375c8-694c-4fa5-8b36-05f50c8e61b2", "74346f6f-c7ee-4506-a204-baf48e13decb"]

In [87]:
# Aggregate data into an index for each set of pools id's
eth_curve_stable_index_df = aggregate_historicals(all_pools_dict, eth_curve_stable_comps_pool_ids, exclude_pool_ids)
eth_convex_stable_index_df = aggregate_historicals(all_pools_dict, eth_convex_stable_comps_pool_ids, exclude_pool_ids)
base_stable_index_df = aggregate_historicals(all_pools_dict, base_stable_comps_pool_ids, exclude_pool_ids)
eth_curve_index_df = aggregate_historicals(all_pools_dict, eth_curve_comps_pool_ids, exclude_pool_ids)
eth_convex_index_df = aggregate_historicals(all_pools_dict, eth_convex_comps_pool_ids, exclude_pool_ids)

In [88]:
# Merge dataframes to keep TVL from Curve and APY from Convex for each index group
eth_curve_stable_index_df.drop('apy', axis=1, inplace=True)
eth_convex_stable_index_df.drop('tvlUsd', axis=1, inplace=True)

eth_curve_index_df.drop('apy', axis=1, inplace=True)
eth_convex_index_df.drop('tvlUsd', axis=1, inplace=True)


eth_stable_index_df = pd.merge(eth_curve_stable_index_df, eth_convex_stable_index_df, on='timestamp', how='inner')
eth_index_df = pd.merge(eth_curve_index_df, eth_convex_index_df, on='timestamp', how='inner')

In [89]:
# Create list to print as header of sheet with index names
index_header = ["Mainnet Stables Index", "Base Stables Index", "Mainnet ETH Index"]

# Concatenate every dataframe into one dataframe
indexed_historicals_df = pd.concat([eth_stable_index_df, base_stable_index_df, eth_index_df], axis = 1, sort=False)

In [90]:
# Insert 2 blank elements between each title in the list for spreadsheet formatting
formatted_index_header = []

for title in index_header:
  formatted_index_header.append(title)
  for i in range(2):
    formatted_index_header.append("")
formatted_index_header.pop()

''

In [91]:
# Select 'Indexed Historicals' tab
indexed_historicals_sheet = lp_landscape.worksheet("Indexed Historicals")

# Clear every cell in the sheet
indexed_historicals_sheet.clear()

# Write formatted list as first row of the 'Pool Historicals' sheet for indexing
indexed_historicals_sheet.update("A1", [formatted_index_header])

# Update data in 'LP Update Historicals' sheet under first row
gd.set_with_dataframe(indexed_historicals_sheet, indexed_historicals_df, row=2,
                      include_index=False, include_column_header=True, resize=True)

/var/folders/vf/7r30jcsd0190g5bqdf7ryl800000gn/T/ipykernel_82881/667794954.py:8: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  indexed_historicals_sheet.update("A1", [formatted_index_header])


### Export Stables and Volatile tabs to csv files

In [92]:
# Extract relevant columns
stables_data = stables_sheet.get('A:U')

# Convert to a DataFrame
stables_df = pd.DataFrame(stables_data[1:], columns=stables_data[0])

# Export to CSV
stables_df.to_csv('stables.csv', index=False)

In [93]:
# Extract relevant columns
volatile_data = volatile_sheet.get('A:U')

# Convert to a DataFrame
volatile_df = pd.DataFrame(volatile_data[1:], columns=volatile_data[0])

# Export to CSV
volatile_df.to_csv('volatile.csv', index=False)